# Exercise

Build a *blast-based* method to predict the presence of BPTI/Kunitz domain
in proteins available in SwissProt using the human proteins as a reference.
• Select all Proteins in SwissProt with BPTI/Kunitz domain.
• Separate human from non human proteins. Use the non human proteins
as a positive in the testing set.
• Generate a random set of negative of the same size of the positive set.
• Remove both positives and negatives from SwissProt and perform the
prediction based on the results of the blast search. 

checking the files we downloaded:

In [4]:
grep '>'  Human_PF00014.fasta |wc

      18     200    1778


      Then we have collected a non human pf00014 file lets check how many sequences it contains.

In [5]:
grep '>' NotHuman_PF00014.fasta |wc

     341    3479   35934


These two sets are used as a positives set. The `Human_PF00014.fasta` as training positive and the the `NotHuman_PF00014.fasta` as testing positive set. And then we have a set that are the negatives. So a set that are only Human but NOT PF00014 kunitz this dataset contains more than 20 000.

In [6]:
 grep '>' Human_NotPF00014.fasta |wc

   20347  222853 2042133


Now if you want to split the dataset into 2 parts. Lets take the first 1000.000:

In [7]:
 grep '>' Human_NotPF00014.fasta |head -n 10000 |wc

   10000  109297 1005583


Then we can find the 'name' or ID of the 100 000st sequence:
By piping;

`head -- display first lines of a file`
`SYNOPSIS
     head [-n count | -c bytes] [file ...]`

`DESCRIPTION
     This filter displays the first count lines or bytes of each of the specified files, or
     of the standard input if no files are specified.  If count is omitted it defaults to
     10.` 
`tail -- display the last part of a file`
` -n number`

So we pipe the head of - n 10000 into | tail -n 1. That way we get the last of the list.

In [8]:
grep '>' Human_NotPF00014.fasta |head -n 10000 |tail -n 1

>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2


Now that we have the name of the 10000st seq - we want to find the in which line number it is located.

In [9]:
grep -n O95081 Human_NotPF00014.fasta 

109065:>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2


The sequence is found in the line `109065`. We can also retrieve the 10th line after the match:

In [11]:
grep -n -A 10 O95081 Human_NotPF00014.fasta #|less

109065:>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2
109066-MVMAAKKGPGPGGGVSGGKAEAEAASEVWCRRVRELGGCSQAGNRHCFECAQRGVTYVDI
109067-TVGSFVCTTCSGLLRGLNPPHRVKSISMTTFTEPEVVFLQSRGNEVCRKIWLGLFDARTS
109068-LVPDSRDPQKVKEFLQEKYEKKRWYVPPDQVKGPTYTKGSASTPVQGSIPEGKPLRTLLG
109069-DPAPSLSVAASTSSQPVSQSHARTSQARSTQPPPHSSVKKASTDLLADIGGDPFAAPQMA
109070-PAFAAFPAFGGQTPSQGGFANFDAFSSGPSSSVFGSLPPAGQASFQAQPTPAGSSQGTPF
109071-GATPLAPASQPNSLADVGSFLGPGVPAAGVPSSLFGMAGQVPPLQSVTMGGGGGSSTGLA
109072-FGAFTNPFTAPAAQSPLPSTNPFQPNGLAPGPGFGMSSAGPGFPQAVPPTGAFASSFPAP
109073-LFPPQTPLVQQQNGSSFGDLGSAKLGQRPLSQPAGISTNPFMTGPSSSPFASKPPTTNPF
109074-L
109075->sp|P32970|CD70_HUMAN CD70 antigen OS=Homo sapiens OX=9606 GN=CD70 PE=1 SV=2


Knowing that one entry 'spans over' 10 lines - line one being the identifier and the remaining ones the sequence, we also know that we have to add 9 lines in order to 'keep' the whole sequence. As not to trucate it keeping only line one of it. The last line shold therefore be 109074.

(Or an easier way to approach it would have been to access sequence 100001 and find the number of the previous line...)

To check again that we calculated it right and we will obtain the entire seq of the sp|095081 we use head again: 

In [12]:
head -n 109074 Human_NotPF00014.fasta |tail -n 10

>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2
MVMAAKKGPGPGGGVSGGKAEAEAASEVWCRRVRELGGCSQAGNRHCFECAQRGVTYVDI
TVGSFVCTTCSGLLRGLNPPHRVKSISMTTFTEPEVVFLQSRGNEVCRKIWLGLFDARTS
LVPDSRDPQKVKEFLQEKYEKKRWYVPPDQVKGPTYTKGSASTPVQGSIPEGKPLRTLLG
DPAPSLSVAASTSSQPVSQSHARTSQARSTQPPPHSSVKKASTDLLADIGGDPFAAPQMA
PAFAAFPAFGGQTPSQGGFANFDAFSSGPSSSVFGSLPPAGQASFQAQPTPAGSSQGTPF
GATPLAPASQPNSLADVGSFLGPGVPAAGVPSSLFGMAGQVPPLQSVTMGGGGGSSTGLA
FGAFTNPFTAPAAQSPLPSTNPFQPNGLAPGPGFGMSSAGPGFPQAVPPTGAFASSFPAP
LFPPQTPLVQQQNGSSFGDLGSAKLGQRPLSQPAGISTNPFMTGPSSSPFASKPPTTNPF
L


This way we have reconfirmed that the index of the last line we plan to grab is correct. Now we will save the sto of the following cmd to Humand_NotPF00014_Training.

In [13]:
head -n 109074 Human_NotPF00014.fasta > Human_NotPF00014_Training.fasta

Then we select the remaining ones as the Test set by just modifying the cmd:

In [17]:
tail -n +109075 Human_NotPF00014.fasta > Human_NotPF00014_Testing.fasta

As always now we check weather the the files 'look' as we expect them to look:

In [18]:
grep '^>' Human_NotPF00014_Training.fasta |wc -l

   10000


In [19]:
grep '^>' Human_NotPF00014_Testing.fasta |wc -l

   10347


Prof Capriotti actually used:

`grep '>' Human_NotPF00014_Testing.fasta |wc
   10347  113556 1036550`

Now we have already formatted the Database:
If you want to format a database
The DB now uses as a reference the human BPTI domain. `formatdb -i <file name> -p T`

Problem on my machine the formatdb is deprecated... instead I had to use:

See: https://www.ncbi.nlm.nih.gov/books/NBK279690/

In [ ]:
makeblastdb -in Human_PF00014.fasta -taxid 9606 -dbtype prot

`Building a new DB, current time: 04/16/2020 18:24:24
New DB name:   /Users/ila/01-Unibo/2_LAB1_2nd/capriotti/repkunitz/Human_PF00014.fasta
New DB title:  Human_PF00014.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 18 sequences in 0.0149059 seconds.`

In [20]:
ls

15L-lab1-17-04_p2.mkv.zip       NotHuman_PF00014.bl6
Human_NotPF00014.fasta          NotHuman_PF00014.bl6.best
Human_NotPF00014_Testing.fasta  NotHuman_PF00014.fasta
Human_NotPF00014_Training.fasta Untitled.ipynb
Human_PF00014.bl6               bl8
Human_PF00014.fasta             kunitz_meeting.rtf
Human_PF00014.fasta.phr         ncbi-blast-2.2.26+.dmg
Human_PF00014.fasta.pin         not_not.fasta
Human_PF00014.fasta.psq


`ila@Immanuelas-Air:~/01-Unibo/2_LAB1_2nd/capriotti/repkunitz$ ls
13L_lab1-15-04_all.mp4          Human_PF00014.fasta             NotHuman_PF00014.fasta
Human_NotPF00014.fasta          Human_PF00014.fasta.phr         ncbi-blast-2.2.26+.dmg
Human_NotPF00014_Testing.fasta  Human_PF00014.fasta.pin         not_not.fasta
Human_NotPF00014_Training.fasta Human_PF00014.fasta.psq`

Now just to check, we will do the blast of the 
human proteins against the Human proteins just as a consistency test.
`#replacing blastpgp from version 2.2.26 (legacy) -i = -query, -d = -db, -o = -out, -m = -outfmt. `


In [22]:
# outfmt 6 is the Tabular form
psiblast -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6

The output was redirected into the `Human_PF00014.bl6` file
looking like that:

In [24]:
head Human_PF00014.bl6

sp|Q06481|APLP2_HUMAN	sp|Q06481|APLP2_HUMAN	100.000	763	0	0	1	763	1	763	0.0	1590
sp|Q06481|APLP2_HUMAN	sp|P05067|A4_HUMAN	51.111	495	178	11	313	762	294	769	1.73e-155	463
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	41.667	84	47	1	288	369	111	194	8.02e-21	84.0
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	40.625	64	34	2	298	360	28	88	2.75e-14	64.7
sp|Q06481|APLP2_HUMAN	sp|P49223|SPIT3_HUMAN	47.170	53	28	0	309	361	35	87	5.02e-16	65.1
sp|Q06481|APLP2_HUMAN	sp|O95925|EPPI_HUMAN	45.283	53	29	0	309	361	76	128	2.48e-14	62.0
sp|Q06481|APLP2_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	50.943	53	26	0	308	360	384	436	3.64e-14	66.2
sp|Q06481|APLP2_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	41.667	36	21	0	325	360	343	378	2.85e-06	40.8
sp|Q06481|APLP2_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	26.271	118	62	6	597	691	188	303	0.88	23.1
sp|Q06481|APLP2_HUMAN	sp|O95428|PPN_HUMAN	31.298	131	82	2	289	417	733	857	4.17e-13	63.5


In [ ]:
# sp|Q06481|APLP2_HUMAN   sp|Q06481|APLP2_HUMAN   100.000 763     0       0       1       763     1       763     0.0     1590
# sp|Q06481|APLP2_HUMAN   sp|P05067|A4_HUMAN      51.111  495     178     11      313     762     294     769     1.73e-155       463
# sp|Q06481|APLP2_HUMAN   sp|O43291|SPIT2_HUMAN   41.667  84      47      1       288     369     111     194     8.02e-21        84.0
# sp|Q06481|APLP2_HUMAN   sp|O43291|SPIT2_HUMAN   40.625  64      34      2       298     360     28      88      2.75e-14        64.7
# sp|Q06481|APLP2_HUMAN   sp|P49223|SPIT3_HUMAN   47.170  53      28      0       309     361     35      87      5.02e-16        65.1

The 11th column shows the e-value. To find the highest (=worst) e-value we use reverse sort by 11th column. `-nrk n- number, r- reverse, k- column`

In [25]:
sort -nrk 11 Human_PF00014.bl8 |head #|less

sp|Q9BQY6|WFDC6_HUMAN	sp|Q02388|CO7A1_HUMAN	48.000	25	11	2	12	35	2315	2338	10.0	16.9
sp|P10646|TFPI1_HUMAN	sp|P12111|CO6A3_HUMAN	45.098	51	28	0	217	267	3112	3162	9.99e-12	56.6
sp|P02760|AMBP_HUMAN	sp|Q9BQY6|WFDC6_HUMAN	59.259	27	11	0	256	282	102	128	9.98e-06	35.8
sp|O95428|PPN_HUMAN	sp|P02760|AMBP_HUMAN	38.462	52	32	0	754	805	231	282	9.96e-10	52.4
sp|Q8TEU8|WFKN2_HUMAN	sp|O43278|SPIT1_HUMAN	54.545	44	20	0	393	436	398	441	9.95e-11	54.7
sp|Q02388|CO7A1_HUMAN	sp|P12111|CO6A3_HUMAN	23.721	215	149	8	1025	1233	12	217	9.92e-06	41.6
sp|Q96NZ8|WFKN1_HUMAN	sp|P10646|TFPI1_HUMAN	29.545	132	59	1	313	410	137	268	9.91e-16	68.9
sp|P12111|CO6A3_HUMAN	sp|Q2UY09|COSA1_HUMAN	24.828	145	96	3	1639	1778	798	934	9.90e-06	41.6
sp|Q8IUA0|WFDC8_HUMAN	sp|P12111|CO6A3_HUMAN	66.667	6	2	0	153	158	2395	2400	9.8	18.1
sp|P12111|CO6A3_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	28.788	66	47	0	2897	2962	266	331	9.8	21.9


We see the worst value belongs to Q9BQY6

In [ ]:
#sp|Q9BQY6|WFDC6_HUMAN   sp|Q02388|CO7A1_HUMAN   48.000  25      11      2       12      35      2315    2338    10.0    16.9

We can also change the evalue to 1000:

In [26]:
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6 -evalue 1000

In [37]:
#sort -nrk 11 Human_PF00014.bl8 |less
sort -nrk 11 Human_PF00014.bl8 |head

sp|Q9BQY6|WFDC6_HUMAN	sp|Q02388|CO7A1_HUMAN	48.000	25	11	2	12	35	2315	2338	10.0	16.9
sp|P10646|TFPI1_HUMAN	sp|P12111|CO6A3_HUMAN	45.098	51	28	0	217	267	3112	3162	9.99e-12	56.6
sp|P02760|AMBP_HUMAN	sp|Q9BQY6|WFDC6_HUMAN	59.259	27	11	0	256	282	102	128	9.98e-06	35.8
sp|O95428|PPN_HUMAN	sp|P02760|AMBP_HUMAN	38.462	52	32	0	754	805	231	282	9.96e-10	52.4
sp|Q8TEU8|WFKN2_HUMAN	sp|O43278|SPIT1_HUMAN	54.545	44	20	0	393	436	398	441	9.95e-11	54.7
sp|Q02388|CO7A1_HUMAN	sp|P12111|CO6A3_HUMAN	23.721	215	149	8	1025	1233	12	217	9.92e-06	41.6
sp|Q96NZ8|WFKN1_HUMAN	sp|P10646|TFPI1_HUMAN	29.545	132	59	1	313	410	137	268	9.91e-16	68.9
sp|P12111|CO6A3_HUMAN	sp|Q2UY09|COSA1_HUMAN	24.828	145	96	3	1639	1778	798	934	9.90e-06	41.6
sp|Q8IUA0|WFDC8_HUMAN	sp|P12111|CO6A3_HUMAN	66.667	6	2	0	153	158	2395	2400	9.8	18.1
sp|P12111|CO6A3_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	28.788	66	47	0	2897	2962	266	331	9.8	21.9


Now you see that 

In [ ]:
#sp|P48307|TFPI2_HUMAN   sp|P05067|A4_HUMAN      27.273  11      8       0       154     164     551     561     999     11.5

This gives us information of the range of e-values we may get. 

If we are not interested in the full matches. We can use awk to check for the id's 
With the cmd below we get all the complete matches.

In [ ]:
#awk '{if($1 == $2)print $0}' Human_PF00014.bl8 |less
#if the first index of a line is equal to the second, print the whole line ($0_)

In [ ]:
`
sp|Q06481|APLP2_HUMAN   sp|Q06481|APLP2_HUMAN   100.000 763     0       0       1       763     1       763     0.0     1590
sp|P05067|A4_HUMAN      sp|P05067|A4_HUMAN      100.000 770     0       0       1       770     1       770     0.0     1610
sp|Q02388|CO7A1_HUMAN   sp|Q02388|CO7A1_HUMAN   100.000 2944    0       0       1       2944    1       2944    0.0     5494
sp|P12111|CO6A3_HUMAN   sp|P12111|CO6A3_HUMAN   100.000 3177    0       0       1       3177    1       3177    0.0     6455
sp|P02760|AMBP_HUMAN    sp|P02760|AMBP_HUMAN    100.000 352     0       0       1       352     1       352     0.0     734
sp|O95925|EPPI_HUMAN    sp|O95925|EPPI_HUMAN    100.000 133     0       0       1       133     1       133     8.42e-100       274
sp|P10646|TFPI1_HUMAN   sp|P10646|TFPI1_HUMAN   100.000 304     0       0       1       304     1       304     0.0     630
sp|O43278|SPIT1_HUMAN   sp|O43278|SPIT1_HUMAN   100.000 529     0       0       1       529     1       529     0.0     1099
sp|O43291|SPIT2_HUMAN   sp|O43291|SPIT2_HUMAN   100.000 252     0       0       1       252     1       252     0.0     521
sp|P48307|TFPI2_HUMAN   sp|P48307|TFPI2_HUMAN   100.000 235     0       0       1       235     1       235     2.53e-180       486
sp|Q8TEU8|WFKN2_HUMAN   sp|Q8TEU8|WFKN2_HUMAN   100.000 576     0       0       1       576     1       576     0.0     1199
sp|Q2UY09|COSA1_HUMAN   sp|Q2UY09|COSA1_HUMAN   100.000 1125    0       0       1       1125    1       1125    0.0     2187
sp|Q96NZ8|WFKN1_HUMAN   sp|Q96NZ8|WFKN1_HUMAN   100.000 548     0       0       1       548     1       548     0.0     1112
sp|O95428|PPN_HUMAN     sp|O95428|PPN_HUMAN     100.000 1278    0       0       1       1278    1       1278    0.0     2586
sp|Q9BQY6|WFDC6_HUMAN   sp|Q9BQY6|WFDC6_HUMAN   100.000 131     0       0       1       131     1       131     1.27e-98        270
sp|Q8IUA0|WFDC8_HUMAN   sp|Q8IUA0|WFDC8_HUMAN   100.000 241     0       0       1       241     1       241     0.0     492
sp|P49223|SPIT3_HUMAN   sp|P49223|SPIT3_HUMAN   100.000 89      0       0       1       89      1       89      4.67e-65        182
sp|Q6UDR6|SPIT4_HUMAN   sp|Q6UDR6|SPIT4_HUMAN   100.000 99      0       0       1       99      1       99      2.56e-72        201
(END)
`

These are obviously the 18 proteins we have in our dataset.

In [38]:
awk '{if ($1 == $2)print $0}' Human_PF00014.bl8 |wc
#       18     216    1468

      18     216    1468


But we rather want to know what happened with the negatives. So lets blast the negatives.

In [39]:
psiblast -query Human_NotPF00014.fasta -db Human_PF00014.fasta -out Human_NotPF00014_Training.bl6 -outfmt 6

We can less the file *.bl6
At this point we want to know the best hit between the negative, that are not Kunitz/BPTI and the Human Kunitz/BPTI. 
Now we sort the output to find the sequence with the lowest e-value in the 11th column.

In [ ]:
#sort -gk 11 Human_NotPF00014_Training.bl8 |less

Now we see something interesting here: there are 2 seq that we have retrieved from a Uniprot search that are not classified as Human BPTI but they have an e-value that is zero.
The first col is Human non BPTI and the second col is Human BPTI that we have in our dataset.


In [ ]:
sp|A6NMZ7|CO6A6_HUMAN   sp|P12111|CO6A3_HUMAN   32.023  2239    1310    43      27      2138    639     2792    0.0     1013
sp|A8TX70|CO6A5_HUMAN   sp|P12111|CO6A3_HUMAN   31.051  2235    1320    51      30      2130    639     2786    0.0     927

This means probably that those two (CO6A6_HUMAN, CO6A5_HUMAN) have a domain that is 
extremely similar to BPTI and has not yet been annotated by pfam.

Then there are many other hits that have an evalue of 10^-3 :
This number is also a possible threshold for selecting **homologues**

`
sp|A6NMZ7|CO6A6_HUMAN   sp|P12111|CO6A3_HUMAN   32.023  2239    1310    43      27      2138    639     2792    0.0     1013
sp|A8TX70|CO6A5_HUMAN   sp|P12111|CO6A3_HUMAN   31.051  2235    1320    51      30      2130    639     2786    0.0     927
sp|A0A075B6J6|LV322_HUMAN       sp|O95428|PPN_HUMAN     25.926  81      40      2       29      108     1053    1114    0.001   28.5
sp|A0A075B6J6|LV322_HUMAN       sp|O95428|PPN_HUMAN     33.750  80      35      4       29      108     919     980     0.001   28.5
sp|A0A075B6T6|TVAL2_HUMAN       sp|O95428|PPN_HUMAN     30.556  72      40      3       32      94      1142    1212    0.001   28.5
sp|A0A0J9YXQ4|PMA6E_HUMAN       sp|Q02388|CO7A1_HUMAN   30.182  275     119     8       86      293     2379    2647    0.001   32.3
sp|A0A1B0GUY1|MRCOL_HUMAN       sp|Q02388|CO7A1_HUMAN   25.397  189     126     5       106     282     2248    2433    0.001   31.2
sp|A0A1B0GUY1|MRCOL_HUMAN       sp|Q02388|CO7A1_HUMAN   33.333  51      34      0       206     256     1294    1344    0.001   31.2
sp|A2VEC9|SSPO_HUMAN    sp|O95428|PPN_HUMAN     36.585  82      41      4       2510    2584    313     390     0.001   35.8
sp|A6NDA9|LRIT2_HUMAN   sp|Q02388|CO7A1_HUMAN   30.851  94      59      4       359     452     864     951     0.001   32.0
sp|A6NF34|ANTRL_HUMAN   sp|Q02388|CO7A1_HUMAN   29.787  94      58      4       172     262     1157    1245    0.001   32.7
sp|B2RNN3|C1T9B_HUMAN   sp|Q02388|CO7A1_HUMAN   39.796  98      35      3       71      156     1296    1381    0.001   31.2
sp|O14498|ISLR_HUMAN    sp|O95428|PPN_HUMAN     29.630  54      38      0       294     347     1169    1222    0.001   31.6
sp|O15232|MATN3_HUMAN   sp|Q02388|CO7A1_HUMAN   23.529  136     98      4       102     234     1072    1204    0.001   32.0
sp|O15394|NCAM2_HUMAN   sp|O95428|PPN_HUMAN     23.333  210     96      8       222     382     921     1114    0.001   32.7
sp|O60271|JIP4_HUMAN    sp|P12111|CO6A3_HUMAN   25.000  88      63      1       487     574     1346    1430    0.001   33.9
sp|O60500|NPHN_HUMAN    sp|Q96NZ8|WFKN1_HUMAN   27.381  84      56      3       740     820     186     267     0.001   33.5
sp|O60575|ISK4_HUMAN    sp|Q8TEU8|WFKN2_HUMAN   40.909  22      13      0       46      67      138     159     0.001   27.7
sp|O60575|ISK4_HUMAN    sp|Q96NZ8|WFKN1_HUMAN   40.909  22      13      0       46      67      120     141     0.001   27.7
sp|O75056|SDC3_HUMAN    sp|O95428|PPN_HUMAN     64.706  17      6       0       405     421     1186    1202    0.001   31.6
sp|O75096|LRP4_HUMAN    sp|O43278|SPIT1_HUMAN   40.000  35      21      0       231     265     335     369     0.001   33.9
`

In [ ]:
sort -gk 11 Human_NotPF00014_Training.bl8 |head -n 2 |awk '{print $1,$2,$11}

Now we may see that there are many non BPTI/Kunitz proteins that get a low e-value. Moving to the training set we can now rund blast on both the positive and the negative sets.

In [3]:
# Run on positive testing set
# psiblast -query NotHuman_PF00014.fasta -db Human_PF00014.fasta -out NotHuman_PF00014.bl8
# didnt work on my version
# blastp -in FASTAIN -db DATABASE - out OUTFILE.bl6 -outfmt 6
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6

In [1]:
cat Human_PF00014.bl6 |wc -l

     835


`NotHuman_PF00014.bl8` contains the alignment of all non-human PF00014 proteins against the Human PF00014 sequences contained in `Human_PF00014.fasta`

Now we will run the same also for the human testing.

In [1]:
blastp -query NotHuman_PF00014.fasta -db Human_PF00014.fasta -out NotHuman_PF00014.bl6 -outfmt 6

In [2]:
cat Human_NotPF00014_2Training.bl6 |wc -l

  114207


`Human_NotPF00014_Testing.bl8` contains the alignment of the 1,000 sequences (Training set) of all human proteins `not belonging to PF00014` against the ones belonging to PF00014 family.


We need first to rank the output on the posotive set of non Human BPTI/Kunitz proteins. To do so we grep based on the identifiers and select the mathch with lowest e-value.

In [ ]:
# idk what he wants but this doenst work for me:
for i in `awk '{print $1}' NotHuman_PF00014.bl8 |sort -u ` ; do grep $i NotHuman_PF00014.bl8 |sort -gk 11 |head -n 1 ; done > NotHuman_PF00014.bl6.best

Let's select an e-value of 0.001 as a classification threshold for the classification of BPTI/kunitz proteins. Accorddin to this assumption we calculated the number of proteins with maximu e-value below that threshold. 